# Notes on the Szegedy Quantum Walks | Introduction to quantum walks

**Table of contents**

0. _Introduction to random walks_
    1. Markov chains and properties
    2. Boltzmann distribution and the Ising model
    4. Metropolis-Hastings algorithm
    5. Python implementation
1. _Introduction to quantum walks_
    1. From Markov chains to unitary quantum walks
    2. Construction of $W$
    3. Python implementation

## From Markov chains to unitary quantum walks

In a Markov chain, the eigenvalues of the transition operator that are close to 1 control the convergence rate, the mixing time, and therefore the difficulty of sampling. For reversible chains, this dependence is especially clean: the slowest nontrivial mode is governed by the spectral gap 
$\gamma = 1 - \lambda_2$. For quantum algorithms, the goal is not to reproduce the classical stochastic dynamics step-by-step, but to preserve and exploit the same spectral information in a form compatible with unitary evolution.

The issue is that stochastic evolution is inherently non-unitary: it is contractive and dissipative, “forgetting” the initial condition as it approaches the stationary distribution. In an eigenmode picture (most cleanly stated for reversible chains, where the relevant operator is diagonalizable in an orthogonal basis), repeated application of the transition matrix damps non-stationary components:
$$p_0 = \pi + \sum_{j \ge 2} c_j v_j 
\qquad\Rightarrow\qquad
P^t p_0 = \pi + \sum_{j \ge 2} c_j \lambda_j^t v_j 
$$
so eigenvalues $\lambda_j \in (-1, 1)$ appears as a contraction factor $\lambda_j^t$. No unitary operator acting on the same state space can mimic this behavior.

Instead of trying to implement the dissipative map 
$P$ as a physical time step, one constructs a unitary operator $W$ acting on an enlarged Hilbert space whose action decomposes into invariant blocks each encoding a singular value of $P$. Concretely (for reversible chains), the classical eigenvalues $\lambda_j \in [-1, 1]$ are mapped to eigenphases of $W$. Thus the classical "slow modes" $\lambda_j \approx 1$ correspond to small angles $\theta_j \approx 0$, and the stationary mode $\lambda_1 = 1$ corresponds to the eigenphase $0$ (a fixed component). 

**We require the MC to be reversible**.

### The operator $X$

The **reversible** $P$ don't need to be Hermitian. This is an issue, as the eigenvectors will not be orthogonal (later failing to define orthogonal invariant subspaces).

We need another operator that has the same spectrum yet a better geometry. Note that $P$ is self-adjoint in the $\pi$-weighted inner product, 
$$\langle f, g \rangle_\pi := \sum_x \pi(x) f(x) \bar g(x),$$
for reversibility (detailed balance) it holds
$$\langle f, Pg \rangle_\pi = \langle Pf, g \rangle_\pi.$$

Therefore we can use the following similarity transform,
$$X := D^{-1} P D, \qquad D = \mathrm{diag}(\sqrt\pi),$$
that ensures the operator $X$ is symmetric, 
$$X_{yx} = \sqrt\pi_y \frac{P_{yx}}{\sqrt \pi_x} = \sqrt\pi_x \frac{P_{xy}}{\sqrt\pi_y} = X_{xy}$$
This means the spectrum of $X$ corresponds to the singular values of $P$, while the eigenvectors are orthogonal. 

(Note: We need $P$ to be reversible as otherwise is that there is no detailed balance and thus no transformation making $P$ self-adjoint.)

### Operator $W$

We define a unitary operator $W$ acting on the Hilbert space
$$
\mathcal H = \mathbb{C}^{|\Omega|} \otimes \mathbb{C}^{|\Omega|}.
$$
For a reversible Markov chain with transition matrix $P$ and stationary distribution $\pi$, let $\{ \lambda_1=1,\lambda_2,\ldots,\lambda_N \}\subset[-1,1]$ the spectrum of $X = D^{-1} P D$.

The Szegedy construction yields a unitary operator $W$ whose spectrum decomposes as
$$
\mathrm{spec}(W) =
\underbrace{\{1\}}_{\text{stationary}}
\;\cup\;
\underbrace{\{ e^{\pm i 2 \arccos(\lambda_j)} \mid j=2,\ldots,N \}}_{\text{classical relaxation modes}}
\;\cup\;
\underbrace{\{\pm 1\}}_{\text{auxiliary / junk}}.
$$

Correspondingly, the Hilbert space admits an orthogonal direct-sum decomposition
$$
\mathcal H
=
\mathcal S_{\mathrm{stat}}
\;\oplus\;
\bigoplus_{j=2}^N \mathcal S_j
\;\oplus\;
\mathcal S_\perp,
$$
with respect to which $W$ acts block-diagonally:
$$
W
=
[1]
\;\oplus\;
\bigoplus_{j=2}^N
\begin{pmatrix}
\cos(2\theta_j) & -\sin(2\theta_j) \\
\sin(2\theta_j) & \cos(2\theta_j)
\end{pmatrix}
\;\oplus\;
W_\perp,
\qquad
\cos\theta_j=\lambda_j.
$$

Each subspace is orthogonal to the others and invariant under $W$. Their interpretation is as follows:

- **$\mathcal S_{\mathrm{stat}}$** is the stationary subspace. It is one-dimensional, associated with the unique eigenvalue $\lambda_1=1$, and $W$ acts as the identity on it.

- **$\mathcal S_j$** is the subspace associated with the classical spectral mode $\lambda_j\in(-1,1)$. It is two-dimensional, and the action of $W$ on $\mathcal S_j$ is a planar rotation by angle $2\arccos(\lambda_j)$.

- **$\mathcal S_\perp$** is the orthogonal complement of the subspace generated by the Markov-chain embedding. It carries no classical information about the chain. For states prepared via the Markov-chain encoding, this subspace is not populated and can therefore be ignored in algorithmic applications; $W$ acts trivially on it.


## Construction of $W$

Recall
$$
\mathcal H = \mathbb{C}^N \otimes \mathbb{C}^N = \mathrm{span} \{ \ket{x}\ket{y} \mid x, y \in \Omega \}. 
$$

We need to have a bit of faith here and follow this construction. 

Define an isometry $V : \mathbb{C}^N \to \mathcal H$ on the basis states by
$$
V \ket{x} = \ket{x} \sum_{y \in \Omega} \sqrt{P_{yx}} \ket{y} =: \ket{\psi_x}.
$$
One can check each $\ket{\psi_x}$ is normalized and that $V^\dagger V = \mathbb{I}$.

The image of $V$ determines a subspace of $\mathcal H$, 
$$
\mathcal A := \mathrm{Im}(V) = \mathrm{span}\{ \ket{\psi_x} \mid x \in \Omega \}, 
\qquad 
\Pi_\mathcal A := V V^\dagger, 
\qquad
R_\mathcal A := 2 \Pi_{\mathcal A} - \mathbb{I}, 
$$ 

Let $S$ be the swap operator such that $S\ket{x}\ket{y} = \ket{y}\ket{x}$. Then define a second subspace,
$$
\mathcal B := S \mathcal A, 
\qquad 
\Pi_{\mathcal B} := S \Pi_{\mathcal A} S, 
\qquad 
R_{\mathcal B} := 2 \Pi_{\mathcal B} - \mathbb{I}.
$$

The walk operator is 
$$
W = R_{\mathcal B} R_{\mathcal A}.
$$

----


We can check this construction will give us the correct unitary. A product of reflections always yields invariant subspaces:
$$
\mathcal S_{\parallel} := \mathcal A + \mathcal B, 
\qquad 
\mathcal S_{\bot} := (\mathcal A + \mathcal B)^\bot.
$$

These spaces are invariant under both $R_\mathcal{A}, R_\mathcal{B}$ and thus invariant under $W$. 

**The action of $W$ onto $\mathcal S_\bot$ is trivial**. In fact, if $\ket{\phi} \in \mathcal S_\bot = (\mathcal A + \mathcal B)^\bot$, then it is orthogonal to any vector in $\mathcal A$ and any vector in $\mathcal B$. Hence both reflections act as $2\cdot 0 - \mathbb I$ and
$$
W \ket\phi 
= R_\mathcal{B} R_\mathcal{A} \ket\phi 
= R_\mathcal{B} (-\ket\phi) 
= - (-\ket\phi) 
= \ket\phi,
$$
so
$$
W|_{\mathcal S_\bot} = \mathbb{I}.
$$

**The action of $W$ onto $\mathcal S_\parallel$ is block diagonal wrt one-dimensional and two-dimensional invariant blocks**.  
The two subspaces $\mathcal A$ and $\mathcal B$ (both of dimension $N$) admit principal angles
$$
0 \le \theta_1 \le \theta_2 \le \cdots \le \theta_N \le \frac{\pi}{2}.
$$
For each principal angle there exist unit vectors $\ket{a_j} \in \mathcal A$ and $\ket{b_j} \in \mathcal B$ such that
$$
\langle a_j | b_j \rangle = \cos\theta_j,
$$
and the planes $\mathrm{span}\{\ket{a_j},\ket{b_j}\}$ are mutually orthogonal.

For each $j$, the subspace
$$
\mathcal S_j := \mathrm{span}\{\ket{a_j},\ket{b_j}\}
$$
is invariant under both reflections, and the action of $W$ on $\mathcal S_j$ is a rotation by angle $2\theta_j$.

**Finally we need to compute these principal angles and relate them to $X$**.  
To compute the principal angles, note that if $A,B$ denote orthonormal basis matrices of $\mathcal A,\mathcal B$, then the singular values of $A^\dagger B$ are $\cos\theta_j$.  
Here $A = V$ and $B = S V$, so
$$
\bra{x} V^\dagger S V \ket{y}
= \braket{\psi_x | S | \psi_y}
= \sqrt{P_{yx} P_{xy}}
= \bra{x} X \ket{y}.
$$
Hence
$$
\lambda_j = \cos\theta_j,
$$
where $\lambda_j$ are the eigenvalues of $X$, and for each $j \ge 2$,
$$
W|_{\mathcal S_j}
=
\begin{pmatrix}
\cos(2\arccos \lambda_j) & -\sin(2\arccos \lambda_j) \\
\sin(2\arccos \lambda_j) & \cos(2\arccos \lambda_j)
\end{pmatrix}.
$$


### Example

Here's a simple example with NumPy.

In [1]:
import numpy as np
from numpy.linalg import eig, svd, norm
np.set_printoptions(precision=6, suppress=True)

We define an reversible but _not_ symmetric column-stochastic $P$,

In [2]:
P = np.array([
    [0.5,      0.5,      0.5     ],
    [0.3,      1/6,      0.5     ],
    [0.2,      1/3,      0.0     ],
], dtype=float)
print("P (column-stochastic, P[y,x]) =\n", P)
print("column sums =", P.sum(axis=0))

P (column-stochastic, P[y,x]) =
 [[0.5      0.5      0.5     ]
 [0.3      0.166667 0.5     ]
 [0.2      0.333333 0.      ]]
column sums = [1. 1. 1.]


Its stationary distribution is the following (follows the column-stochastic convention: $P[y,x] = Pr(\text{next}=y \mid \text{current}=x)$).

In [3]:
pi = np.array([0.5, 0.3, 0.2], dtype=float)
N = len(pi)
print("pi =", pi)

pi = [0.5 0.3 0.2]


We can check the detailed balance holds:

In [4]:
print("||P pi - pi|| =", norm(P @ pi - pi))

||P pi - pi|| = 0.0


We check the similarity transformation ensuring $P$ becomes symmetric.

In [5]:
D_sqrt  = np.diag(np.sqrt(pi))
D_isqrt = np.diag(1.0/np.sqrt(pi))
X = D_isqrt @ P @ D_sqrt
print("X = D^{-1/2} P D^{1/2} =\n", X)
print("||X - X^T|| =", norm(X - X.T))

X = D^{-1/2} P D^{1/2} =
 [[0.5      0.387298 0.316228]
 [0.387298 0.166667 0.408248]
 [0.316228 0.408248 0.      ]]
||X - X^T|| = 1.9229626863835638e-16


The spectrum of $X$ is:

In [6]:
evals, U = eig(X)
evals = np.real_if_close(evals, tol=1e-10).astype(float)
order = np.argsort(evals)[::-1]
evals = evals[order]
print("Ordered spectrum of X =", evals)

Ordered spectrum of X = [ 1.        0.       -0.333333]


The operator $V$ is constructed as follows:

In [7]:
# V (N^2 x N): columns are |x> ⊗ sum_y sqrt(P_{y x}) |y>
V = np.column_stack([
    np.concatenate([np.zeros(x*N), np.sqrt(P[:, x]), np.zeros((N-1-x)*N)])
    for x in range(N)
])

The swap operator $S$ is constructed as follows:

In [8]:
# Swap S on C^N ⊗ C^N
S = np.zeros((N*N, N*N))
for x in range(N):
    for y in range(N):
        S[y*N + x, x*N + y] = 1.0

Sanity checks:

In [17]:
print("If V an isometry?")
print("||V^T V - I|| =", norm(V.T @ V - np.eye(N)))
print("If V^dag S V = X?")
print("||V^T S V - X|| =", norm(V.T @ S @ V - X))

If V an isometry?
||V^T V - I|| = 2.220446049250313e-16
If V^dag S V = X?
||V^T S V - X|| = 1.5700924586837752e-16


We now construct $W$:

In [28]:
Pi_A = V @ V.T
Pi_B = S @ Pi_A @ S
I = np.eye(N*N)
R_A = 2*Pi_A - I
R_B = 2*Pi_B - I
W = R_B @ R_A

We find the principal angles of subspaces $\mathcal A, \mathcal B$, which are related to the eigenvalues of $X$:

In [19]:
# SVD step: singular values of X are cos(principal angles) in [0,1]
U_svd, svals, Vt_svd = svd(X)
print("Singular values of X (cos principal angles) =", svals)

Singular values of X (cos principal angles) = [1.       0.333333 0.      ]


We can then find the blocks in $W$:

In [20]:
# Print exact 1D/2D rotation blocks in the principal-angle basis
print("Exact 1D/2D rotation blocks in the principal-angle basis:")
for j in range(N):
    lam = float(evals[j])
    u = U[:, j]
    a = V @ u
    a = a / norm(a)
    b = S @ a

    if abs(1 - abs(lam)) < 1e-10:
        block = np.array([[float(a @ (W @ a))]])
        print(f"\nBlock j={j+1}, lambda={lam:.6f} (1D)")
        print(block)
        continue

    theta = float(np.arccos(np.clip(lam, -1, 1)))

    # principal-angle basis on span{a,b}: e1=a, e2=(b - lam a)/sqrt(1-lam^2)
    e1 = a
    e2 = (b - lam*e1) / np.sqrt(1 - lam**2)

    B = np.column_stack([e1, e2])
    block = B.T @ W @ B

    R = np.array([[np.cos(2*theta), -np.sin(2*theta)],
                  [np.sin(2*theta),  np.cos(2*theta)]])

    print(f"\nBlock j={j+1}, lambda={lam:.6f}, theta=arccos(lambda)={theta:.6f} (2D)")
    print("block =\n", block)
    print("Is the block the expected rotation? =", norm(block - R) < 1e-8)

# Eigenvalues of W
evalsW = eig(W)[0]
evalsW = evalsW[np.argsort(np.angle(evalsW))]
print("\nEigenvalues of W (sorted by angle) = ")
for lam_j in evalsW:
    print(np.real_if_close(np.round(lam_j, 5)), end="   ")

Exact 1D/2D rotation blocks in the principal-angle basis:

Block j=1, lambda=1.000000 (1D)
[[1.]]

Block j=2, lambda=0.000000, theta=arccos(lambda)=1.570796 (2D)
block =
 [[-1. -0.]
 [ 0. -1.]]
Is the block the expected rotation? = True

Block j=3, lambda=-0.333333, theta=arccos(lambda)=1.910633 (2D)
block =
 [[-0.777778  0.628539]
 [-0.628539 -0.777778]]
Is the block the expected rotation? = True

Eigenvalues of W (sorted by angle) = 
(-0.77778-0.62854j)   1.0   1.0   1.0   1.0   1.0   (-0.77778+0.62854j)   -1.0   -1.0   

## Python implementation

A quick-and-dirty python/Qiskit implementation of $W$ is made available here. 

It is way easier to visualize an example of $P$ with states being a power of two.

In [1]:
import numpy as np
from numpy.linalg import eig, svd, norm
np.set_printoptions(precision=6, suppress=True)
from qiskit.circuit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator
from szegedymcmc.gates.szegedy_v import SzegedyV
from szegedymcmc.gates.szegedy_swap import SzegedySwap
from szegedymcmc.gates.szegedy_reflection_i0 import SzegedyReflectionI0
from szegedymcmc.gates.szegedy_w import SzegedyW

N = 2

P = np.array([
    [0.3, 0.6],
    [0.7, 0.4]
])

The class `SzegedyV` takes as input any reversible and irreducible $P$ and creates the corresponding state preparation $V$ unitary in the form of a quantum circuit. 

Notably, the implementation is based on the `StatePreparation` primitive of Qiskit. This syntetizes the unitary $V$ up to arbitrary precision albeit the cost is polynomial in the number of entries (exponential in the number of qubits). A better implementation is discussed later.


In [23]:
V = np.column_stack([
    np.concatenate([np.zeros(x*N), np.sqrt(P[:, x]), np.zeros((N-1-x)*N)])
    for x in range(N)
])
print("V (isometry)")
print(V.shape, "\n", V)

V_circuit = SzegedyV(P)
V_unitary = Operator(V_circuit).data
print("V (quantum circuit)")
print(V_unitary.shape, "\n", V_unitary)

# check that V (isometry) matches the restriction of the unitary to B=0 inputs
# permutation that maps Qiskit's row index (y*N + x) -> your row index (x*N + y)
perm = np.array([ (i % N) * N + (i // N) for i in range(N * N) ], dtype=int)

# pick the |B=0> columns in Qiskit ordering: columns x (not x*N)
ok = all(np.allclose(V[:, x], V_unitary[perm, x]) for x in range(N))

print("Does V match V_unitary (accounting for basis ordering)?", ok)

V (isometry)
(4, 2) 
 [[0.547723 0.      ]
 [0.83666  0.      ]
 [0.       0.774597]
 [0.       0.632456]]
V (quantum circuit)
(4, 4) 
 [[ 0.547723+0.j  0.      +0.j -0.83666 +0.j  0.      +0.j]
 [ 0.      +0.j  0.774597+0.j  0.      +0.j -0.632456+0.j]
 [ 0.83666 +0.j  0.      +0.j  0.547723+0.j  0.      +0.j]
 [ 0.      +0.j  0.632456+0.j  0.      +0.j  0.774597+0.j]]
Does V match V_unitary (accounting for basis ordering)? True


The class `SzegedyW` takes as input any reversible and irreducible $P$ and creates the walk operator. It uses `SzegedyV`, the `Swap` operator, the `ReflectionI0` operator, too, which are part of the same package. 

In [5]:
S = np.zeros((N*N, N*N))
for x in range(N):
    for y in range(N):
        S[y*N + x, x*N + y] = 1.0

S_gate = SzegedySwap(N)
S_unitary = np.real_if_close(Operator(S_gate).data)
print("Is the implementation of Swap gate correct?", np.allclose(S, S_unitary))

Is the implementation of Swap gate correct? True


In [6]:
# Expected reflection R_{I0} = I_A ⊗ (2|0><0| - I)_B
# In Qiskit's basis ordering with A as least-significant register:
# R_{I0} matrix is (2|0><0| - I)_B ⊗ I_A
I_A = np.eye(N)
proj0 = np.zeros((N, N))
proj0[0, 0] = 1.0
Rb = 2.0 * proj0 - np.eye(N)  # +1 on |0>, -1 on all others
R = np.kron(Rb, I_A)          # B is the "most significant" register in Qiskit's ordering

R_gate = SzegedyReflectionI0(N)
R_unitary = np.real_if_close(Operator(R_gate).data)
ok = np.allclose(R, R_unitary) or np.allclose(R, -R_unitary) # Some implementations differ by a global phase (-1). Accept either.
print("Is the implementation of ReflectionI0 gate correct?", ok)

Is the implementation of ReflectionI0 gate correct? True


In [22]:
Pi_A = V @ V.T
Pi_B = S @ Pi_A @ S
I = np.eye(N*N)
R_A = 2*Pi_A - I
R_B = 2*Pi_B - I
W = R_B @ R_A

W_gate = SzegedyW(P)
W_unitary = np.real_if_close(Operator(W_gate).data)

evals_W = eig(W)[0]
evals_W = evals_W[np.argsort(np.angle(evals_W))]
print("Eigenvalues of W build via the definition (sorted by angle) = ")
for lam_j in evals_W:
    print(np.real_if_close(np.round(lam_j, 5)), end="   ")

evals_W_qc = eig(W_unitary)[0]
evals_W_qc = evals_W_qc[np.argsort(np.angle(evals_W_qc))]
print("\nEigenvalues of W from qiskit circuit (sorted by angle) = ")
for lam_j in evals_W_qc:
    print(np.real_if_close(np.round(lam_j, 5)), end="   ")

evals_P = eig(P)[0]
evals_P = evals_P[np.argsort(np.angle(evals_P))]
phases_P = np.exp(1j*2*np.arccos(evals_P))
print("\nPhases related to the eigenvalues of P, i.e. exp(i*2*arccos(lam_j))")
for phi_j in phases_P:
    print(np.real_if_close(np.round(phi_j, 5)), end="   ")


# Eigenvalues of W build via the definition (sorted by angle) = 
# (-0.82-0.57236j)   1.0   1.0   (-0.82+0.57236j)   
# Eigenvalues of W from qiskit circuit (sorted by angle) = 
# 1.0   1.0   -1.0   -1.0   

Eigenvalues of W build via the definition (sorted by angle) = 
(-0.82-0.57236j)   1.0   1.0   (-0.82+0.57236j)   
Eigenvalues of W from qiskit circuit (sorted by angle) = 
(-0.82-0.57236j)   1.0   1.0   (-0.82+0.57236j)   
Phases related to the eigenvalues of P, i.e. exp(i*2*arccos(lam_j))
1.0   (-0.82-0.57236j)   